# Random Predictor

In [ ]:
import random
import os

import mlflow
from mlflow.pyfunc.model import PythonModel

from sklearn.metrics import f1_score

from data_utils import get_train_test_split_for_stock
from config import *

## Define a custom model

In [ ]:
# Define a custom model
class RandomPredictor(PythonModel):
    """
    Docstring here...
    """
    
    def __init__(self):
        pass

    def predict(self, context, model_input):
        return model_input.apply(lambda column: random.randint(0, 1), axis=1)

In [ ]:
MODEL_FOLDER_NAME = "random_predictor"
PATH_TO_MODEL_FOLDER = os.path.join(PATH_TO_CUSTOM_MODELS, MODEL_FOLDER_NAME)
PATH_TO_MODEL_FOLDER

In [ ]:
# Cleaning (the folder must not exist)
#os.chdir(PATH_TO_NOTEBOOKS)
#os.getcwd()

In [ ]:
#!rm -fr /home/ksatola/work/data/custom_models/random_predictor

## Save the model

In [ ]:
# Construct and save the model

r = RandomPredictor()

mlflow.pyfunc.save_model(path=PATH_TO_MODEL_FOLDER, python_model=r)

## Load the model

In [ ]:
# Load the model in `python_function` format
cls = mlflow.pyfunc.load_model(PATH_TO_MODEL_FOLDER)

## Predict

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_split_for_stock(PATH_TO_DATA_FILE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
model_output = cls.predict(X_test)
model_output.head()

In [ ]:
mlflow.set_experiment('SP_EXP_Modelling')

with mlflow.start_run(run_name='random_predictor'):
    
    f1_max = 0
    
    for i in range(20):
    
        model_output = cls.predict(X_test)
        
        mlflow.log_metric("Predictions sum", model_output.sum())
        
        y_pred = model_output
        f1 = f1_score(y_test, y_pred)
    
        mlflow.set_tag("Model_name", "Random Predictor")
        mlflow.log_metric(key="testing_f1_score", value=f1)

        mlflow.set_tag("Model_name", "random predictor")
        
        mlflow.log_artifact("100_EXP_Predictor_random.ipynb")
        mlflow.log_artifact(PATH_TO_MODEL_FOLDER)
        
        f1_max = max(f1_max, f1)
        print(f1)
    
        # Use to show error and failed MLflow experiment
        #mlflow.log_artifact("100_Predictor_random2.ipynb")
        
    print(f"Best F1-score for randon predictor: {f1_max}")

# Show
- predictions metric diagram in mlflow
- show failed experiment